In [1]:
from langchain_google_genai import GoogleGenerativeAI
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_community.utilities import SQLDatabase

api_key  = os.getenv('GOOGLE_API_KEY')


In [2]:
llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key=api_key)
print(
    llm.invoke(
        "India is a?"
    )
)

sovereign state


In [5]:
db_user="root"
db_password="12345"
db_host="localhost"
db_name="world"
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info = 3)
print(db.table_info)


CREATE TABLE city (
	`ID` INTEGER NOT NULL AUTO_INCREMENT, 
	`Name` CHAR(35) NOT NULL DEFAULT '', 
	`CountryCode` CHAR(3) NOT NULL DEFAULT '', 
	`District` CHAR(20) NOT NULL DEFAULT '', 
	`Population` INTEGER NOT NULL DEFAULT '0', 
	PRIMARY KEY (`ID`), 
	CONSTRAINT city_ibfk_1 FOREIGN KEY(`CountryCode`) REFERENCES country (`Code`)
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from city table:
ID	Name	CountryCode	District	Population
1	Kabul	AFG	Kabol	1780000
2	Qandahar	AFG	Qandahar	237500
3	Herat	AFG	Herat	186800
*/


CREATE TABLE country (
	`Code` CHAR(3) NOT NULL DEFAULT '', 
	`Name` CHAR(52) NOT NULL DEFAULT '', 
	`Continent` ENUM('Asia','Europe','North America','Africa','Oceania','Antarctica','South America') NOT NULL DEFAULT 'Asia', 
	`Region` CHAR(26) NOT NULL DEFAULT '', 
	`SurfaceArea` DECIMAL(10, 2) NOT NULL DEFAULT '0.00', 
	`IndepYear` SMALLINT, 
	`Population` INTEGER NOT NULL DEFAULT '0', 
	`LifeExpectancy` DECIMAL(3, 1), 
	`GNP` DECIMAL(10, 2

In [13]:
from langchain.chains import create_sql_query_chain
sql_chain = create_sql_query_chain(llm, db)
resp=sql_chain.invoke({"question":"how many countries have the Life Expectancy  greater than 75 give me list of countries names with their population count?"})
resp
new_ans=db.run("SELECT Name, Population FROM world.country WHERE LifeExpectancy > 75")
new_ans

"[('Aruba', 103000), ('Anguilla', 8000), ('Andorra', 78000), ('Argentina', 37032000), ('American Samoa', 68000), ('Australia', 18886000), ('Austria', 8091800), ('Belgium', 10239000), ('Bermuda', 65000), ('Canada', 31147000), ('Switzerland', 7160400), ('Chile', 15211000), ('Costa Rica', 4023000), ('Cuba', 11201000), ('Cayman Islands', 38000), ('Cyprus', 754700), ('Germany', 82164700), ('Denmark', 5330000), ('Spain', 39441700), ('Finland', 5171300), ('France', 59225700), ('Faroe Islands', 43000), ('United Kingdom', 59623400), ('Gibraltar', 25000), ('Guadeloupe', 456000), ('Greece', 10545700), ('French Guiana', 181000), ('Guam', 168000), ('Hong Kong', 6782000), ('Ireland', 3775100), ('Iceland', 279000), ('Israel', 6217000), ('Italy', 57680000), ('Jamaica', 2583000), ('Jordan', 5083000), ('Japan', 126714000), ('Kuwait', 1972000), ('Libyan Arab Jamahiriya', 5605000), ('Liechtenstein', 32300), ('Luxembourg', 435700), ('Macao', 473000), ('Monaco', 34000), ('Malta', 380200), ('Northern Mariana

In [17]:
## Few Shot Learning:

few_shot=[
    {
        'Question':"how many countries have the Life Expectancy  greater than 75 give me list of countries names with their population count?",
        'SQLQuery':"SELECT Name, Population FROM world.country WHERE LifeExpectancy > 75",
        'SQLResult':"Result of SQL Query",
        "Answer":new_ans
        
    }
]